<a href="https://colab.research.google.com/github/DanB1421/DATA602/blob/main/Brilliant_Problem_Set_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preamble
This problem set is an extension of Problem Set 6.  You will need the following artifacts from last week:

* The MNIST 784 dataset from OpenML, with dimensionality reduced to about 75\%.
* The Support Vector Machine classifier.
* Recoded target variables, such that the target variable is 1 if the digit is less than 5, and 0 otherwise.

(You may copy these artifacts from the posted solutions if needed.)  As with last week, please use the first 60,000 observations as training data, and the remaining 10,000 images as test data.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from joblib import dump, load
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
#fetch OpenML data
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

#split into test/training sets
N=60000
X_train, y_train = X[:N, :], y[:N]
X_test, y_test = X[N:, :], y[N:]

#scale data
scaler = MinMaxScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

/usr/local/lib/python3.9/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
from sklearn.decomposition import PCA # imports PCA module from sklearn
pca = PCA(n_components=0.75) # generates a PCA that includes 75% of the original variance
X_train_sc = pca.fit_transform(X_train_sc) # fits and transforms X_train_sc using PCA
X_test_sc = pca.transform(X_test_sc) # transforms X_test_sc using PCA
print(X_train_sc.shape) # prints the shape of X_train_sc

(60000, 34)


In [ ]:
X_train = pca.fit_transform(X_train) # fits and transforms X_train using PCA
X_test = pca.transform(X_test) # transforms X_test using PCA
print(X_train.shape) # prints the shape of X_train_sc

(60000, 34)


In [ ]:
y_train_mod = y_train.astype(int) # converts y_train to integers
y_test_mod = y_test.astype(int) # converts y_test to integers
y_train_recode = np.where(y_train_mod < 5, 1, 0) # recodes y_train to binary where 1 replaces values less than 5 and 0 replaces all other values
y_test_recode = np.where(y_test_mod < 5, 1, 0) # recodes y_test to binary where 1 replaces values less than 5 and 0 replaces all other values

In [ ]:
from sklearn.svm import SVC # imports SVC module
clf_svc = SVC(C=464.15888336127773,
          break_ties=False,
          cache_size=200,
          class_weight=None,
          coef0=0.0,
          decision_function_shape='ovr',
          degree=3,
          gamma=0.4444444444444444,
          kernel='rbf',
          max_iter=-1,
          probability=False,
          random_state=None,
          shrinking=True,
          tol=0.001,
          verbose=False) # creates SVC using parameters obtained from the search in Problem Set 6 and stores as a classifier

dump(clf_svc, 'clf_svc.pkl', compress=1) # saves the model in a file called clf_svc.pkl

['clf_svc.pkl']

In [ ]:
clf_svc = load('clf_svc.pkl') # loads saved clf_svc model from file
y_train_pred = cross_val_predict(clf_svc, X_train_sc, y_train_recode) # generates cross-validated estimates for each data point using the developed model on the training data
print(classification_report(y_train_recode, y_train_pred)) # prints classification report

              precision    recall  f1-score   support

           0       0.98      0.96      0.97     29404
           1       0.96      0.98      0.97     30596

    accuracy                           0.97     60000
   macro avg       0.97      0.97      0.97     60000
weighted avg       0.97      0.97      0.97     60000



# Problem 1 -- Classifiers

Construct 3 classifiers using different algorithms, not including the SVM model built last week, that classify the MNIST dataset with an $F_1$ score of at least 0.9.  At least one classifier must use gradient boosting (AdaBoost, Gradient Boost, or xgboost).  Show the $F_1$ score and classification report for each model.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier # imports AdaBoost module from sklearn
clf_ada = AdaBoostClassifier(n_estimators=2000,
                             random_state=0,
                             learning_rate=0.1) # creates AdaBoost classifier

In [ ]:
y_train_pred = cross_val_predict(clf_ada, X_train, y_train_recode) # generates cross-validated estimates for each data point using the developed model on the training data
print(classification_report(y_train_recode, y_train_pred)) # prints classification report

              precision    recall  f1-score   support

           0       0.84      0.87      0.86     29404
           1       0.87      0.84      0.86     30596

    accuracy                           0.86     60000
   macro avg       0.86      0.86      0.86     60000
weighted avg       0.86      0.86      0.86     60000



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier # imports GradientBoostingClassifier module from sklearn
clf_grd = GradientBoostingClassifier(n_estimators=1000, random_state=0,
                                     max_depth=8,
                                     max_features='sqrt') # creates GradientBoostingClassifier

In [ ]:
y_train_pred = cross_val_predict(clf_grd, X_train, y_train_recode) # generates cross-validated estimates for each data point using the developed model on the training data
print(classification_report(y_train_recode, y_train_pred)) # prints classification report


              precision    recall  f1-score   support

           0       0.97      0.98      0.98     29404
           1       0.98      0.97      0.98     30596

    accuracy                           0.98     60000
   macro avg       0.98      0.98      0.98     60000
weighted avg       0.98      0.98      0.98     60000



In [ ]:
import xgboost as xgb # imports xgb module from xgboost
clf_xgb = xgb.XGBClassifier(n_estimators=1000,
                            max_depth=8,
                            colsample_bylevel=0.5) # creates xgboost classifier

In [ ]:
y_train_pred = cross_val_predict(clf_xgb, X_train, y_train_recode) # generates cross-validated estimates for each data point using the developed model on the training data
print(classification_report(y_train_recode, y_train_pred)) # prints classification report


              precision    recall  f1-score   support

           0       0.97      0.98      0.98     29404
           1       0.98      0.98      0.98     30596

    accuracy                           0.98     60000
   macro avg       0.98      0.98      0.98     60000
weighted avg       0.98      0.98      0.98     60000



In [ ]:
from sklearn.linear_model import LogisticRegression # imports LogisticRegression module from sklearn
clf_log = LogisticRegression(C=.1, max_iter=100) # creates LogisticRegression classifier

In [ ]:
y_train_pred = cross_val_predict(clf_log, X_train_sc, y_train_recode) # generates cross-validated estimates for each data point using the developed model on the training data
print(classification_report(y_train_recode, y_train_pred)) # prints classification report

              precision    recall  f1-score   support

           0       0.80      0.82      0.81     29404
           1       0.82      0.81      0.81     30596

    accuracy                           0.81     60000
   macro avg       0.81      0.81      0.81     60000
weighted avg       0.81      0.81      0.81     60000



In [ ]:
from sklearn.tree import DecisionTreeClassifier # imports DecisionTreeClassifier module from sklearn
clf_decision_tree = DecisionTreeClassifier(ccp_alpha=.01) # creates DecisionTreeClassifier

In [ ]:
y_train_pred = cross_val_predict(clf_decision_tree, X_train, y_train_recode) # generates cross-validated estimates for each data point using the developed model on the training data
print(classification_report(y_train_recode, y_train_pred)) # prints classification report

              precision    recall  f1-score   support

           0       0.73      0.76      0.74     29404
           1       0.76      0.73      0.74     30596

    accuracy                           0.74     60000
   macro avg       0.74      0.74      0.74     60000
weighted avg       0.74      0.74      0.74     60000



In [ ]:
from sklearn.ensemble import RandomForestClassifier # imports RandomForestClassifier module from sklearn
clf_random_forest = RandomForestClassifier(n_estimators=1000,
                             oob_score=True,
                             min_samples_split=10,
                             min_samples_leaf=5) # creates RandomForestClassifier

In [ ]:
y_train_pred = cross_val_predict(clf_random_forest, X_train, y_train_recode) # generates cross-validated estimates for each data point using the developed model on the training data
print(classification_report(y_train_recode, y_train_pred)) # prints classification report

              precision    recall  f1-score   support

           0       0.95      0.97      0.96     29404
           1       0.97      0.95      0.96     30596

    accuracy                           0.96     60000
   macro avg       0.96      0.96      0.96     60000
weighted avg       0.96      0.96      0.96     60000



In [ ]:
dump(clf_grd, 'clf_grd.pkl', compress=1) # saves the model in a file called clf_grd.pkl
dump(clf_xgb, 'clf_xgb.pkl', compress=1) # saves the model in a file called clf_xgb.pkl
dump(clf_random_forest, 'clf_random_forest.pkl', compress=1) # saves the model in a file called clf_random_forest.pkl

['clf_random_forest.pkl']

# Problem 2 -- Voting ensemble model

Build a voting ensemble model that combines the three classifiers from the previous problem, in addition to the SVM model developed last week.  What is the $F_1$ score of the ensemble model?

In [ ]:
clf_svc = load('clf_svc.pkl') # loads clf_svc from file
from sklearn.pipeline import make_pipeline # imports make_pipeline module from sklearn

clf_svc_pipeline = make_pipeline(scaler, clf_svc) # makes a pipeline that runs a scaler and the clf_svc model
dump(clf_svc_pipeline, 'clf_svc_pipeline.pkl', compress=1) # saves the model in a file called clf_svc_pipeline.pkl

['clf_svc_pipeline.pkl']

In [ ]:
clf_grd = load('clf_grd.pkl') # loads clf_grd from file
clf_xgb = load('clf_xgb.pkl') # loads clf_xgb from file
clf_random_forest = load('clf_random_forest.pkl') # loads clf_random_forest from file
clf_svc_pipeline = load('clf_svc_pipeline.pkl') # loads clf_svc_pipeline from file

In [ ]:
from sklearn.ensemble import VotingClassifier # imports VotingClassifier module from sklearn
clfs = [('gbm', clf_grd),
        ('xgbm', clf_xgb),
        ('random forest', clf_random_forest),
        ('svm', clf_svc_pipeline)] # creates list of tuples containing classifiers to pass through VotingClassifier

clf_vot = VotingClassifier(clfs) # creates VotingClassifier from list of tuples containing classifiers
dump(clf_vot, 'clf_vot.pkl', compress=1) # saves the model in a file called clf_svc_pipeline.pkl

['clf_vot.pkl']

In [ ]:
clf_vot = load('clf_vot.pkl') # loads clf_vot from file
y_train_pred = cross_val_predict(clf_vot, X_train, y_train_recode, cv=2) # generates cross-validated estimates for each data point using the developed model on the training data
print(classification_report(y_train_recode, y_train_pred)) # prints classification report

              precision    recall  f1-score   support

           0       0.99      0.96      0.98     29404
           1       0.96      0.99      0.98     30596

    accuracy                           0.98     60000
   macro avg       0.98      0.98      0.98     60000
weighted avg       0.98      0.98      0.98     60000



The F1 score is about 0.98

## Problem 3 -- Stacking ensemble model
Stacking uses a final classifier (often a logistic regression) that outputs an aggregate of the predictors. Repeat the previous problem using a `StackingClassifier` rather than voting to compute the final prediction.  What is the $F_1$ score of the stacking classifier?


In [ ]:
from sklearn.ensemble import StackingClassifier # imports StackingClassifier from sklearn
clf_stack = StackingClassifier(clfs) # creates StackingClassifier from list of tuples containing classifiers
dump(clf_stack, 'clf_stack.pkl', compress=1) # saves the model in a file called clf_stack.pkl

['clf_stack.pkl']

In [ ]:
clf_stack = load('clf_stack.pkl') # loads clf_stack from file
y_train_pred = cross_val_predict(clf_stack, X_train, y_train_recode, cv=2) # generates cross-validated estimates for each data point using the developed model on the training data
print(classification_report(y_train_recode, y_train_pred)) # prints classification report

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     29404
           1       0.98      0.98      0.98     30596

    accuracy                           0.98     60000
   macro avg       0.98      0.98      0.98     60000
weighted avg       0.98      0.98      0.98     60000



The F1 score is about 0.98.

## Problem 4 -- Evaluation

At this point in the assignment, you have six classifiers:

* the support vector classifier from last week,
* the three classifiers from problem 1,
* the voting classifier from problem 2, and
* the stacking classifier from problem 3

Identify the model with the highest $F_1$ score, and train this model with the full training dataset.  Finally, score the test data against this model.  Does the model demonstrate predictive validity (i.e., are the $F_1$ scores for the test data comparable to the training data)?

In [ ]:
f1_grd = (2 * (0.97 * 0.98) / (0.97 + 0.98)) # uses precision and recall values from classification report to calculate approximate f1 score of clf_grd
print(f1_grd) # prints f1_grd score

0.974974358974359


In [ ]:
f1_xgb = (2 * (0.98 * 0.98) / (0.98 + 0.98)) # uses precision and recall values from classification report to calculate approximate f1 score of clf_xgb
print(f1_xgb) # prints f1_xgb score

0.98


In [ ]:
f1_vot = (2 * (0.99 * 0.96) / (0.99 + 0.96)) # uses precision and recall values from classification report to calculate approximate f1 score of clf_vot
print(f1_vot) # prints f1_vot score

0.9747692307692307


In [ ]:
f1_stack = (2 * (0.98 * 0.98) / (0.98 + 0.98)) # uses precision and recall values from classification report to calculate approximate f1 score of clf_stack
print(f1_stack) # prints f1_stack score

0.98


In [ ]:
clf_xgb = load('clf_xgb.pkl') # loads clf_xgb from file

In [ ]:
clf_xgb.fit(X_train, y_train_recode) # fits clf_xgb to the training dataset

SVC(C=464.15888336127773, gamma=0.4444444444444444)

In [ ]:
y_test_pred = cross_val_predict(clf_xgb, X_test, y_test_recode) # generates cross-validated estimates for each data point using the developed model on the test data
print(classification_report(y_test_recode, y_test_pred)) # prints classification report

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4861
           1       0.51      1.00      0.68      5139

    accuracy                           0.51     10000
   macro avg       0.26      0.50      0.34     10000
weighted avg       0.26      0.51      0.35     10000



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


According to the above evaluation, the xgboost model does not have predictive validity.